In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable, Type
import abc

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
from torch import Tensor


from models.ridge_ALOOCV import fit_ridge_ALOOCV
from models.sandwiched_least_squares import sandwiched_LS_dense, sandwiched_LS_diag, sandwiched_LS_scalar

In [2]:
# Make regression data X, y
N = 1000
N_test = 1000
D = 10
d = 2
X = torch.randn(N, D)
X_test = torch.randn(N_test, D)
w_true = torch.randn(D, d)
y = (X @ w_true)**2 + torch.randn(N, d) * 0.1  # Adding some noise
y_test = (X_test @ w_true)**2 + torch.randn(N_test, d) * 0.1  # Adding some noise

In [3]:
from old.models import RidgeCVModule

#dense      
model = RidgeCVModule(
        lower_alpha=1e-6,
        upper_alpha=1e6,
        n_alphas=10,
     )

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(15.4181) std tensor(1.0053e-06)
test rmse tensor(15.8648) std tensor(1.0053e-06)
train tensor([15.4181, 15.4181, 15.4181, 15.4181, 15.4181, 15.4181, 15.4181, 15.4181,
        15.4181, 15.4181])
test tensor([15.8648, 15.8648, 15.8648, 15.8648, 15.8648, 15.8648, 15.8648, 15.8648,
        15.8648, 15.8648])


In [54]:
from models.random_feature_boosting import GreedyRFBoostRegressor

model = GreedyRFBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=512,
        randfeat_x0_dim=512,
        l2_reg=0.1,
        l2_ghat=0.001,
        boost_lr=1.0,
        sandwich_solver="dense",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(5):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(0.5208) std tensor(0.0157)
test rmse tensor(1.4119) std tensor(0.0530)
train tensor([0.5097, 0.5011, 0.5402, 0.5228, 0.5302])
test tensor([1.3475, 1.4079, 1.4519, 1.3759, 1.4765])


In [5]:
# train rmse tensor(2.3312) std tensor(0.5462)
# test rmse tensor(3.3370) std tensor(0.6041)
# train tensor([2.5459, 1.9770, 2.1338, 3.6567, 1.9788, 2.5264, 2.1827, 1.9025, 2.6022,
#         1.8058])
# test tensor([3.7197, 2.8087, 2.9656, 4.6065, 2.9207, 3.3118, 3.1447, 3.0470, 4.0548,
#         2.7906])

In [6]:
from models.random_feature_boosting import GreedyRFBoostRegressor

model = GreedyRFBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.01,
        l2_ghat=0.0001,
        boost_lr=1.0,
        sandwich_solver="diag",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(4.3488) std tensor(0.3546)
test rmse tensor(5.7388) std tensor(0.5112)
train tensor([4.4656, 3.9879, 4.2363, 4.4247, 4.4911, 3.5892, 4.3382, 4.4682, 4.5911,
        4.8959])
test tensor([5.6735, 5.3793, 5.4282, 5.7887, 6.1953, 4.6483, 5.7691, 6.1447, 5.8929,
        6.4685])


In [7]:
from models.random_feature_boosting import GreedyRFBoostRegressor

model = GreedyRFBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.01,
        l2_ghat=0.0001,
        boost_lr=1.0,
        sandwich_solver="scalar",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(12.6554) std tensor(0.0016)
test rmse tensor(13.8479) std tensor(0.0018)
train tensor([12.6556, 12.6555, 12.6552, 12.6559, 12.6550, 12.6556, 12.6512, 12.6570,
        12.6570, 12.6559])
test tensor([13.8486, 13.8485, 13.8471, 13.8480, 13.8478, 13.8475, 13.8431, 13.8493,
        13.8492, 13.8494])


In [50]:
from models.random_feature_boosting import GradientRFBoostRegressor

model = GradientRFBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=512,
        randfeat_x0_dim=512,
        l2_reg=0.1,
        l2_ghat=0.000001,
        boost_lr=1.0,
        feature_type="SWIM",
        upscale_type="iid",
    )

results = []
for i in range(5):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(0.5189) std tensor(0.0006)
test rmse tensor(1.1643) std tensor(0.0758)
train tensor([0.5185, 0.5196, 0.5183, 0.5187, 0.5193])
test tensor([1.2378, 1.0961, 1.2547, 1.1072, 1.1256])


In [ ]:
# from old.models import GradientRFBoostRegressor

# model = GradientRFBoostRegressor(
#      hidden_dim=128, 
#      randfeat_xt_dim=512, 
#      randfeat_x0_dim=512,
#      n_layers=5, 
#      feature_type="SWIM", 
#      upscale="dense", 
#      )

# results = []
# for i in range(10):
#     model.fit(X, y)
#     out = model(X)
#     out_test = model(X_test)
#     rmse = torch.sqrt(nn.functional.mse_loss(out, y))
#     rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
#     results.append(torch.tensor([rmse, rmse_test]))
# results = torch.stack(results)
# print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
# print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
# print("train", results[:, 0])
# print("test", results[:, 1])

train rmse tensor(0.6617) std tensor(0.0463)
test rmse tensor(1.5217) std tensor(0.0757)
train tensor([0.7035, 0.5931, 0.7167, 0.6517, 0.6372, 0.7311, 0.6287, 0.6797, 0.6674,
        0.6079])
test tensor([1.5882, 1.4314, 1.6341, 1.5020, 1.4991, 1.6404, 1.4503, 1.4768, 1.5366,
        1.4577])


# End2End

In [10]:
from old.models import End2EndMLPResNet

model = End2EndMLPResNet(
        in_dim=D,
        hidden_dim=128,
        bottleneck_dim=32,
        out_dim=d,
        n_blocks=3,
        loss = "mse",
        lr = 0.1,
        n_epochs = 30,
        end_lr_factor= 0.1,
        weight_decay = 0.001,
        batch_size = 64,
        )

results = []
for i in range(5):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

100%|██████████| 30/30 [00:02<00:00, 13.82it/s]

train rmse tensor(1.8853) std tensor(1.3225)
test rmse tensor(2.6152) std tensor(0.9681)
train tensor([1.0172, 3.1286, 0.8292, 3.5199, 0.9316])
test tensor([2.0540, 3.1394, 2.0177, 4.0727, 1.7921])
